### ライブラリのインポート

In [8]:
import pandas as pd
import numpy as np
import os
import requests
import zipfile

import warnings
warnings.simplefilter('ignore')

### データセットの作成

### MovieLens100 small ファイルをデータセットのダウンロード

In [9]:
url = "http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
filename = url.split('/')[-1]
r = requests.get(url, stream=True)
with open(filename, 'wb') as f:
    for chunk in r.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)
            f.flush()

### ダウンロードしたMovieLens100 small zipファイルの解凍

In [10]:
PATH = os.getcwd() + "/"

In [11]:
zfile_name = 'ml-latest-small'
zfile = zipfile.ZipFile(PATH+ zfile_name + ".zip")
zfile.extractall(PATH)

In [12]:
movies = pd.read_csv(PATH + zfile_name + "/movies.csv")
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [13]:
rating = pd.read_csv(PATH + zfile_name + "/ratings.csv")
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [14]:
dataset = pd.merge(rating, movies, on="movieId")
dataset = dataset[["userId", "title", "rating"]]
dataset

,userId,title,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5
...,...,...,...
100831,610,Bloodmoon (1997),2.5
100832,610,Sympathy for the Underdog (1971),4.5
100833,610,Hazard (2005),3.0
100834,610,Blair Witch (2016),3.5


In [15]:
df = dataset.pivot_table(index='title', columns='userId', values='rating')
df.columns.name = None
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.5,NaN,NaN
xXx (2002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,2.0
xXx: State of the Union (2005),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5


### ユーザー同士の類似値の算定

In [16]:
def get_similairty(person1, person2, dataset):

    # レートがNaNになっているものを除いた、指定した人の見た映画リスト
    movie1 = dataset[dataset[person1].notnull()][person1].index
    movie2 = dataset[dataset[person2].notnull()][person2].index

    ## 各映画とも見た人の集合を取る
    set_movie1 = set(movie1)
    set_movie2 = set(movie2)
    set_both = set_movie1.intersection(set_movie2)

    if len(set_both)==0: #共通でみた映画がない場合は類似度を0とする
        return 0

    # 同じ映画のレビュー点の差の2乗を計算
    # この数値が大きいほど「似ていない」と定義 
    distance = pow(dataset.loc[list(set_both)][person1] - dataset.loc[list(set_both)][person2], 2)
    distance = distance.sum()

    return 1/(1+np.sqrt(distance)) #各映画の似てなさの合計の逆比的な指標を返す

In [17]:
# ユーザーは 1～610
get_similairty(1, 50, df)

0.12002728245132872

### レコメンド関数の実装

In [18]:
def get_recommend(person, top_N, dataset):

     #推薦度スコアを入れるための箱を作っておく
    totals = {}
    simSums = {}

    # 自分以外のユーザのリストを取得してFor文を回す
    # -> 各人との類似度、及び各人からの（まだ本人が見てない）映画の推薦スコアを計算するため
    list_others = list(dataset)
    list_others.remove(person)

    for other in list_others:

        # 本人以外が見た映画
        other_movie = dataset[dataset[other].notnull()][other].index
        set_other = set(other_movie) # 集合

        # 本人が見た映画
        self_movie = dataset[dataset[person].notnull()][person].index        
        set_self_movie = set(self_movie)  # 集合

        # 本人がまだ見たことが無い映画の集合を取得
        set_new_movie = set_other.difference(set_self_movie)

        # あるユーザと本人の類似度を計算(simは0~1の数字)
        sim = get_similairty(person, other, dataset)

        # (本人がまだ見たことがない)映画のリストでFor分を回す
        for movie in set_new_movie:

            # "類似度 x レビュー点数" を推薦度のスコアとして、全ユーザで積算する
            totals.setdefault(movie,0)
            totals[movie] += dataset[other][movie]*sim

            # またユーザの類似度の積算値をとっておき、これで上記のスコアを除する
            simSums.setdefault(movie,0)
            simSums[movie] += sim

    rankings = [(total/simSums[movie],movie) for movie, total in totals.items()]
    rankings.sort()
    rankings.reverse()

    return [i[1] for i in rankings][:top_N]

In [19]:
# ユーザーは 1～610
recommend_movie = get_recommend(12, 5, df)
recommend_movie

['Enter the Void (2009)',
 "Won't You Be My Neighbor? (2018)",
 'Vampire in Venice (Nosferatu a Venezia) (Nosferatu in Venice) (1986)',
 'Vagabond (Sans toit ni loi) (1985)',
 'Unfaithfully Yours (1948)']

In [20]:
pd.DataFrame(recommend_movie, columns=["RecommendMovie"])

,RecommendMovie
0,Enter the Void (2009)
1,Won't You Be My Neighbor? (2018)
2,Vampire in Venice (Nosferatu a Venezia) (Nosfe...
3,Vagabond (Sans toit ni loi) (1985)
4,Unfaithfully Yours (1948)
